In [13]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict
from pydantic import BaseModel, Field
from google import genai
import os

In [14]:
#Creating the object of GEMINI Model 
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [ ]:
# Define a node that calls Gemini
def gemini_node(state: State) -> State:
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=state["question"]
    )
    state["answer"] = response.text
    return state

In [15]:
class EvaluateSchema(BaseModel):

    feedback: str = Field(description="Detailed feedback for the essay")
    score: int = Field(description='Score out of 10', ge=0, le=10)



In [16]:
from google.genai.types import GenerateContentConfig
import json

# Convert Pydantic schema to JSON schema
schema = EvaluateSchema.model_json_schema()

# Create structured model with generation config
config = GenerateContentConfig(
    response_mime_type="application/json",
    response_schema=schema
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Evaluate this essay: [Your essay here]",
    config=config
)

# Parse and validate with Pydantic
result = EvaluateSchema.model_validate_json(response.text)
print(f"Feedback: {result.feedback}")
print(f"Score: {result.score}")

ClientError: 403 PERMISSION_DENIED. {'error': {'code': 403, 'message': 'Your API key was reported as leaked. Please use another API key.', 'status': 'PERMISSION_DENIED'}}